## generate flattened view for biospec database
Only required to be run if a disease is added.

In [ ]:
import os

import pandas
from biospecdb.util import to_bool

os.environ.setdefault('DJANGO_SETTINGS_MODULE', "biospecdb.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
from django.db import connection
from django.db.utils import OperationalError

django.setup()

In [ ]:
def query(sql):
    with connection.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        if not result:
            return
        columns = [col[0] for col in cursor.description]
        return [dict(zip(columns, row)) for row in result]
    
def drop_view(view):
    try:
        query(f"drop view {view}")
    except OperationalError:
        pass

def update_view(view, sql, check=True):
    drop_view(view)
    query(sql)
    if check:
        query(f"select * from {view} limit 1")

In [ ]:
view = "v_symptoms"
symptoms_view = f"""
CREATE VIEW {view} AS
SELECT s.visit_id,
       s.id AS symptom_id,
       d.id AS disease_id,
       d.name AS disease,
       d.value_class,
       s.disease_value,
       s.days_symptomatic,
       s.severity,
       s.disease_value
FROM uploader_symptom s
JOIN uploader_disease d ON d.id=s.disease_id
"""

update_view(view, symptoms_view)

In [ ]:
diseases = query("select * from uploader_disease")

In [ ]:
d = []
# this code should be updated once the is_symptomatic field is gone and disease_value is used for all "diseases"
for disease in diseases:
    if disease["value_class"] == "FLOAT":
        c = 'cast(disease_value as real)'
    elif disease["value_class"] in "STR":
        c = 'disease_value'
    else:
        c = "cast(disease_value AS tinyint)"
    d.append(f"max(case when disease = '{disease['name']}' then {c} else null end) as [{disease['name']}]")

d = "\n,      ".join(d)

view = "v_visit_symptoms"
flattening_view = f"""
create view {view} as
select visit_id
,      {d} 
  from v_symptoms 
 group by visit_id
"""
print(flattening_view)
update_view(view, flattening_view)

In [ ]:
# must (re)define or at least refresh the full_patient view as it depends on the view above that is being updated
view = "full_patient"
full_patient_view = f"""
create view {view} as 
select p.patient_id, p.gender, v.patient_age
,      bs.sample_type, bs.sample_processing, bs.freezing_temp, bs.thawing_time
,      i.spectrometer, i.atr_crystal
,      sd.spectra_measurement, sd.acquisition_time, sd.n_coadditions, sd.resolution, sd.data
,      vs.*
  from uploader_patient p
  join uploader_visit v on p.patient_id=v.patient_id
  join uploader_biosample bs on bs.visit_id=v.id
  join uploader_spectraldata sd on sd.bio_sample_id=bs.id
  join uploader_instrument i on i.id=sd.instrument_id
  left outer join v_visit_symptoms vs on vs.visit_id=v.id
"""
print(full_patient_view)
update_view(view, full_patient_view)

In [ ]:
# clean all
drop_view("full_patient")
drop_view("v_visit_symptoms")
drop_view("v_symptoms")